# Julia implementation of Manuel's code

In [ ]:
using Plots
using ForwardDiff
using SkeelBerzins
using DifferentialEquations

In [ ]:
a = 6

function free_energy_2w(u)

    return u * log(u) + (1 - u) * log(1 - u) + a * u * (1 - u)
end

dFdu = (u) -> ForwardDiff.derivative(free_energy_2w, u)

d2Fdu2 = (u) -> ForwardDiff.derivative(dFdu, u)


In [ ]:
eps = 1e-3

u = range(eps, 1-eps, 200)

F = free_energy_2w.(u)

f = dFdu.(u)
fp = d2Fdu2.(u)

plot(u,  F)
plot!(u, f * eps^0.5)
plot!(u, fp * eps)

## Skeel-Berzins implementation of Cahn-Hilliard

We have the set of equations

$$
\frac{\partial c}{\partial t} = \nabla \cdot (M(c) \cdot \nabla \mu) \\
\mu = f(c) - \gamma \nabla^2 c
$$

After a variable transform $y = \ln(\frac{c}{1-c})$, we get the equations

$$
\frac{dc}{dy}\frac{\partial y}{\partial t} = \nabla \cdot (M(c(y)) \cdot \nabla \mu) \\
\mu = \tilde f(y) + y - \gamma \nabla \left( \frac{dc}{dy} \nabla y \right)
$$

For the specific tranform, we have

$$
\frac{dc}{dy} = c(1-c) = \frac{e^y}{(1+e^y)^2}
$$

and 

$$
f(c) = \tilde f(y) + y
$$

In radial coordinates, we have

$$
\frac{dc}{dy}\frac{\partial y}{\partial t} = \frac{1}{r^2} \partial_r (r^2 M(c(y)) \partial_r \mu) \\
\mu = \tilde f(y) + y - \gamma \frac{1}{r^2} \partial_r \left( r^2 \frac{dc}{dy} \partial_r y \right)
$$


For the Skeel-Berzins integrator, we have

$$
\hat m = 2 \\
\hat c = \left[\frac{dc}{dy}, 0 \right]^T \\
\hat f(r, t, (y, \mu), \partial_r (y, \mu)) = \left[M(c(y)) \partial_r \mu, - \gamma \frac{dc}{dy} \partial_r y\right] \\
\hat s(r, t, (y, \mu), \partial_r (y, \mu)) = [0, \tilde f(y)  + y - \mu]
$$

In [ ]:
# Define the mesh

N_x = 100

R = 1
T = 10

r_mesh = collect(range(0, R; length=N_x));

In [ ]:
gamma = 1

mobility = (c) -> c * (1 - c)

m = 2

# y_of_u = (u) -> log(u / (1 - u))
# u_of_y = (y) -> exp(y) / (1 + exp(y))

# y_of_u = (u) -> u
# u_of_y = (y) -> y

dudy = (y) -> ForwardDiff.derivative(u_of_y, y)
du2dy2 = (y) -> ForwardDiff.derivative(dudy, y)

ft = (u) -> a * u * (1 - u)

function pdefun(r, t, u_state, dudx)

    # Unpack variables.
    y = u_state[1]
    mu = u_state[2]

    dydr = dudx[1]
    dmudr = dudx[2]

    # physical concentration.
    u = u_of_y(y)

    # Construct the Skeel-Berzkins functions.
    c = SVector(dudy.(y), 0)

    f = SVector(mobility.(u) * dmudr,
                - gamma * dydr * dudy.(y))
    s = SVector(0,
                (y + ft.(u) - mu))

    return c, f, s
end

# test the function evaluation
pdefun(0.5, 0.5, SVector(0.5, 0.5), SVector(0.5, 0.5))

In [ ]:
function icfun(x)

    u00 = eps .+ 0. * x

    u0 = SVector(y_of_u.(u00), dFdu.(u00))

    # u0 = SVector(0., 0.)

    return u0
end

# Test the function e.valuation
icfun(r_mesh)

In [ ]:
function bdfun(rl, u_state_l, r_r, u_state_r, t)

    pl = SVector(0, 0)
    ql = SVector(1, 1)  # Neumann-0 BC at inner radius

    pr = SVector(0.1, 0)
    qr = -SVector(1, 1)

    return pl, ql, pr, qr
end

In [ ]:
params_diffEq = SkeelBerzins.Params(; solver=:DiffEq)

t_span = (0, T)

pb = pdepe(m, pdefun, icfun, bdfun, r_mesh, t_span; params=params_diffEq)

In [ ]:
problem = DifferentialEquations.ODEProblem(pb)
sol_diffEq = DifferentialEquations.solve(problem, Rosenbrock23(), abstol=1e-9)
sol_reshaped_diffEq = reshape(sol_diffEq, pb)

In [ ]:
plot(r_mesh, u_of_y.(sol_reshaped_diffEq.u[end][1, :]))
# plot!(r_mesh, sol_reshaped_diffEq.u[end][2, :])